In [1]:
import cv2
import numpy as np

In [12]:
cap = cv2.VideoCapture('cars.mp4')

# background->delete
bgsubmog=cv2.bgsegm.createBackgroundSubtractorMOG()

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))

min_w=80
min_h=80

# detection line
line_h=300


offset = 5
def get_center(x,y,w,h):
    return x+w/2,y+h/2

car_num=0

while True:
    ret,frame = cap.read()
    if ret:
        cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        # gussian filter
        blur=cv2.GaussianBlur(frame,(3,3),5)
        # background
        mask=bgsubmog.apply(blur)


        # erode small noise
        erode = cv2.erode(mask,kernel)
        # after get rid of small -> dilet
        dilate = cv2.dilate(erode,kernel,iterations=2)

        # close, get ride of small piece inside
        close = cv2.morphologyEx(dilate,cv2.MORPH_CLOSE,kernel,iterations=3)

        # get outlook,contours
        cnts, h=cv2.findContours(close,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


        # draw line
        cv2.line(frame,(10,line_h),(600,line_h),(255,0,0),4)

        
        centers=[]

        for (i,c) in enumerate(cnts):
            # draw rectangle
            (x,y,w,h)=cv2.boundingRect(c)

            # get cars
            if(w<min_w or h<min_h):
                continue
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
            center=get_center(x,y,w,h)
            centers.append(center)

        for (x,y) in centers:
            # if between detection line
            if((y>line_h-offset) and (y<line_h+offset)):
                car_num+=1

        cv2.putText(frame,"Cars number:"+str(car_num),(20,20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
        cv2.imshow('a',frame)

    key=cv2.waitKey(1)
    if(key==27):
        break
cap.release()
cv2.destroyAllWindows()
